In [2]:
import pandas as pd
import numpy as np

In [12]:
# read csv in to file
credits = pd.read_csv("credits.csv")

In [13]:
# create a director,actor dictionary with name as key, vlaue in the form [list of movie, number of movies acted ,average rating] and output as a new csv

MOVIE_ID = 0
MOVIE_COUNT = 1
AVG_SCORE = 2

actors_dict = {}
directors_dict = {}
for index, data in credits.iterrows():
    if(data["role"] == "ACTOR"):
        if(data["name"] not in actors_dict.keys()):
            actors_dict[data["name"]] = [[data["id"]],1,0]
        else:
            actors_dict[data["name"]][MOVIE_COUNT] += 1
            actors_dict[data["name"]][MOVIE_ID].append(data["id"])
    else:
        if(data["name"] not in directors_dict.keys()):
            directors_dict[data["name"]] = [[data["id"]],0,1]
        else:
            directors_dict[data["name"]][MOVIE_COUNT] += 1
            directors_dict[data["name"]][MOVIE_ID].append(data["id"])


In [ ]:
# Put all the data into csv file
act_df = pd.DataFrame.from_dict(actors_dict)
act_df = act_df.transpose()
# add title to the data
column_titles = ['movie acted', 'num_of_movie', "avg_score"]  # Replace with desired column titles
act_df.columns = column_titles
act_df.index.name = "name"
act_df.to_csv("actor.csv")

#do exact same thing for director
dir_df = pd.DataFrame.from_dict(directors_dict)
dir_df = dir_df.transpose()
# add title to the data
column_titles = ['movie acted', 'num_of_movie', "avg_score"]  # Replace with desired column titles
dir_df.columns = column_titles
dir_df.index.name = "name"
dir_df.to_csv("director.csv")

calculate weighted score use max-min normalize method to adjust all the valuje from imdb and tmdb then apply weight to each of them

In [11]:
movies = pd.read_csv("titles.csv")

In [17]:
movies_array = np.array((movies["tmdb_score"]))
# Calculate min and max values
min_val = np.min(movies_array)
max_val = np.max(movies_array)

# Perform Max-Min normalization
normalized_data = (movies_array - min_val) / (max_val - min_val)
print(max_val, min_val)
movies["test_score"] = normalized_data
movies.to_csv("test_score.csv", index=False)

0          NaN
1        8.179
2        7.300
3        7.811
4        7.600
         ...  
5845       NaN
5846       NaN
5847     6.300
5848    10.000
5849    10.000
Name: tmdb_score, Length: 5850, dtype: float64
nan nan
